# Prepare PNG segementation masks from COCO

This code processes COCO-formatted annotations and images in a specific directory, modifies some information, and generates corresponding mask images in a PNG file format based on object segmentations.

More information about segmentation mask requirements can be found at https://support.landing.ai/landinglens/docs/upload-labeled-images-seg#segmentation-mask

## Required libraries and packages

In [ ]:
import os
import json
import cv2
import numpy as np
from pycocotools.coco import COCO

## Preparation

### Variables

input_dir: Directory where the images and annotations are stored.
output_dir: Directory where the processed output (mask files and modified annotations) will be stored.

In [ ]:
input_dir = './myProject/input_dir'
output_dir = './myProject/output_dir'

In [ ]:
# This ensures that the output directory exists. If it doesn’t, it creates the directory.
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Loading COCO annotations

This opens and loads the COCO annotations from the file '_annotations.coco.json' located in the input_dir. COCO annotations contain image metadata, categories (labels), and object annotations (bounding boxes, segmentation masks, etc.).

In [ ]:
with open(os.path.join(input_dir, '_annotations.coco.json'), 'r') as f:
    coco_data = json.load(f)

### Modifying the category and annotation IDs
This increments the category_id of each annotation by 1. This is done to re-map the categories to new IDs.

The new dictionary called defect_map, maps the new category_ids to their corresponding names. A mapping for category "0" is predefined as "ok", and the loop appends additional mappings by increasing the id of each category and associating it with its name. 

The value for “0” must be “ok”. “0” is used to identify the background; in other words, areas that do not have classes applied to them. The value “ok” will not be created as a class. If you don’t include this line, you will get an error when you try to upload the images.

In [ ]:
increment = 1

for ann in coco_data['annotations']:
    ann['category_id'] += increment

defect_map = {0:"ok",}
for ann in coco_data['categories']:
    ann['id'] += increment
    defect_map[ann['id']] = ann['name']

### Saving modified annotations and category mapping

The modified annotations (with incremented category_ids) are saved to a new JSON file (_annotations_modified.coco.json). The defect_map (category ID to name mapping) is saved as defect_map.json in the output_dir.

In [ ]:
with open(os.path.join(input_dir, '_annotations_modified.coco.json'), 'w') as f:
    json.dump(coco_data, f)

with open(os.path.join(output_dir, 'defect_map.json'), 'w') as f:
    json.dump(defect_map, f)

### Loading modified annotations

The COCO class is instantiated with the modified annotations, allowing further image and annotation manipulation.

In [ ]:
coco = COCO(os.path.join(input_dir, '_annotations_modified.coco.json'))

## Processing each image

First retrieve a list of all image IDs in the modified COCO dataset. 

Then for each image, the code fetches the image metadata (img_info), loads the image file in grayscale mode using OpenCV, and retrieves its height and width.

### Creating the mask

An empty mask image (a 2D array) with the same height and width as the input image in intialized. Initially, the mask is filled with zeros (black).

### Applying the segmentation to the mask

For each image, load all the annotations (object segments) associated with it.

For each annotation, retrieve the category_id, label (name), and segmentation data. Then, convert the segmentation to a binary mask using annToMask, and updates the corresponding area of the main mask with the object mask.

### Saving the mask
Save the generated mask for each image as a PNG file in the output_dir.

In [ ]:
img_ids = coco.getImgIds()

for img_id in img_ids:
    img_info = coco.loadImgs(img_id)[0]
    img_file = os.path.join(input_dir, img_info['file_name'])
    img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape[:2]

    mask = np.zeros((height, width), dtype=np.uint8)

    for ann in anns:
        label = coco.loadCats(ann['category_id'])[0]['name']
        segmentation = ann['segmentation']

        seg_mask = coco.annToMask(ann)
        mask[seg_mask == 1] = 1

    mask_file = os.path.splitext(img_info['file_name'])[0] + '.png'
    mask_path = os.path.join(output_dir, mask_file)
    cv2.imwrite(mask_path, mask)